In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datetime
from pathlib import Path

import plotly.express as px

import tensorflow as tf

from tqdm.notebook import tqdm

from importlib import reload
import keras_model_creator
import sentinel_utils
import plot_utils
from data_generator import DataGenerator

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
# sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']
# shards_dir = Path.home().joinpath('sentinel_data', 'shards')
# indices = [sentinel_bands.index(b) for b in ['B3', 'B8', 'B6', 'B11']]
# shards = list(shards_dir.joinpath('features_201706').glob('feature*.npy'))
# for shard in tqdm(shards):
#     data = np.copy(np.load(shard))
#     np.save(
#         shards_dir.joinpath('features_2017', shard.stem),
#         data[..., indices]
#     )

In [3]:
reload(sentinel_utils)

model_dir = Path('models', 'selected_model')
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_classes = utils.get_processed_labels()

data_summary = utils.get_data_summary(
    shards_dir, selected_classes, overwrite_existing=False
)

loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32

model_parent_dir = Path('models')
model_dir = model_parent_dir.joinpath(
    f'{loss}-{len(selected_classes.index)}'
    f'-{selected_classes.shape[1]}-{batch_size}-{base_filters}'
)
model_dir.mkdir(parents=True, exist_ok=True)

params = dict(
    loss=loss,
    base_filters=base_filters,
    shards_dir=shards_dir,
    selected_classes=selected_classes,
    data_summary=data_summary,
    batch_size=64,
    dropout=0.5,
    model_dir=model_dir,
    epochs=30,
    overwrite=False,
    verbose=1,
    print_log=0
)

In [ ]:
reload(keras_model_creator)
model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()

Epoch 11/30


I0000 00:00:1722537992.436603   13847 service.cc:145] XLA service 0x7f8f70002590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722537992.436653   13847 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1722538004.820807   13847 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4979 - auc: 0.9047 - loss: 0.3236 - macrof1score: 0.4606 - microf1score: 0.6813 - prc: 0.7691 - precision: 0.7588 - recall: 0.6181 - weightedf1score: 0.6339
Epoch 11: val_recall improved from 0.50000 to 0.63638, saving model to models/binary_crossentropy-237212-7-64-32/model.keras
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 243s 64ms/step - accuracy: 0.4979 - auc: 0.9047 - loss: 0.3236 - macrof1score: 0.4606 - microf1score: 0.6813 - prc: 0.7691 - precision: 0.7588 - recall: 0.6181 - weightedf1score: 0.6339 - val_accuracy: 0.5288 - val_auc: 0.9200 - val_loss: 0.3000 - val_macrof1score: 0.4900 - val_microf1score: 0.7036 - val_prc: 0.8049 - val_precision: 0.7867 - val_recall: 0.6364 - val_weightedf1score: 0.6581 - learning_rate: 2.5000e-04
Epoch 12/30
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5007 - auc: 0.9057 - loss: 0.3217 - macrof1score: 0.4626 - microf1score: 0.6834 - prc: 0.7719 - precision: 0.7584 - recall: 0.6219 - weigh

In [ ]:
model.summary()